In [1]:
import tensorflow as tf

In [2]:
EPOCHS = 5 # hyper parameter

## Network 구조

In [3]:
# keras모델을 '상속'해서 사용

class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.d1 = tf.keras.layers.Dense(128, activation='sigmoid')
        self.d2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

## 학습 루프 정의

In [4]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

In [5]:
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터 불러오기

In [6]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0 

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 모델 인스턴스

In [9]:
model = MyModel()

## 손실함수와 최적화 함수 설정

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

## Performance 측정법 설정

In [12]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


## 학습 진행

In [13]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss,train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100, test_loss.result(), test_accuracy.result() * 100))

Epoch 1, Loss: 0.14844512939453125, Accuracy: 95.72166442871094, Test Loss: 0.1356634497642517, Test Accuracy: 95.80000305175781
Epoch 2, Loss: 0.1148669645190239, Accuracy: 96.68833923339844, Test Loss: 0.11637339740991592, Test Accuracy: 96.59000396728516
Epoch 3, Loss: 0.0922698974609375, Accuracy: 97.38333129882812, Test Loss: 0.09837791323661804, Test Accuracy: 97.0999984741211
Epoch 4, Loss: 0.07573072612285614, Accuracy: 97.86833190917969, Test Loss: 0.09243518859148026, Test Accuracy: 97.18000030517578
Epoch 5, Loss: 0.06342635303735733, Accuracy: 98.22166442871094, Test Loss: 0.08576806634664536, Test Accuracy: 97.39999389648438
